In [ ]:
import plotly.express as px
import pandas as pd

In [ ]:
#---------------------------------------------------
#
# Config *** TO UPDATE ***
#
#---------------------------------------------------

# *** TO UPDATE: change the team number with your own team number
TEAM_NUMBER = "TeamDemo"

# File with AOIS definitions 
AOIS_DEFINITION_FILE = "raw-data/"+TEAM_NUMBER+"/aois definitions/aois.csv"

# This file should be automatically generated from "5. AOI Visits"
DWELLS_FILE = "data/dwells.csv"

In [ ]:
#----------------------------------------------------------------------------------------
#
# 1. AOI Sequence Charts
#
#----------------------------------------------------------------------------------------

In [ ]:
# Read aois definition
aois_df = pd.read_csv(AOIS_DEFINITION_FILE)

In [ ]:
# Display aois definition
display(aois_df)

In [ ]:
# get list of AOIs to plot
aoisToPlot = aois_df["AOI"].tolist()
print(f'AOIs: {aoisToPlot}')

In [ ]:
# Read dwells data using pandas library
data = pd.read_csv(DWELLS_FILE)

# set display.max_columns to none, to show all the columns when using head()
pd.set_option('display.max_columns', None)

In [ ]:
#Preview dwell data
data.head()

In [ ]:
#plot AOI Sequences

#Keep only visits within the aoisToPlot
dataToPlot = data[data['VisitedAOI'].isin(aoisToPlot)].copy(deep=True)

#specify format of 'Dwell Start' and 'Dwell End'
dataToPlot.loc[:,'Dwell Start'] = pd.to_datetime(dataToPlot['Dwell Start'], unit='ms')
dataToPlot.loc[:,'Dwell End'] = pd.to_datetime(dataToPlot['Dwell End'], unit='ms')


In [ ]:
color_discrete_map = None

# Iteratate over the participants and tasks and plot AOI sequences
for participant in dataToPlot['Respondent'].unique():
    
    for task in dataToPlot['SourceStimuliName'].unique():
    
        # Filter in the participant and task data
        participantTaskData = dataToPlot[(dataToPlot['Respondent']==participant) & (dataToPlot['SourceStimuliName']==task)]

        # Plot AOI sequences for the participant data
        fig = None
        # if no colors are assigned yet for the different AOI sequences
        if color_discrete_map is None:
            # Generate a figure with default colors
            fig = px.timeline(participantTaskData, x_start="Dwell Start", x_end="Dwell End", y="VisitedAOI", color="VisitedAOI",  category_orders={"VisitedAOI": [value for value in aoisToPlot]}, title=f'Participant: {participant}, Task: {task}')
            # copy the colors to color_discrete_map in order to reuse them for the other AOI sequence charts
            color_discrete_map = {data.name: data.marker.color for data in fig.data}
        else:
            # Generate a figure with the existing colors in color_discrete_map
            fig = px.timeline(participantTaskData, x_start="Dwell Start", x_end="Dwell End", y="VisitedAOI", color="VisitedAOI",  category_orders={"VisitedAOI": [value for value in aoisToPlot]},  color_discrete_map=color_discrete_map, title=f'Participant: {participant}, Task: {task}')

        fig.update_xaxes(
            tickformat="%Q",  # date format
        ) 

        fig.update_layout(autosize=True,height=600)

        fig.show()